#Part 1: Explore and cluster the neighborhoods in Toronto.
-----

In [2]:
import pandas as pd
import numpy as np

import requests
website_url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

In [4]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [5]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.head()
df.shape

(289, 3)

####Remove rows where Borough=None (Basically the first row.)

In [6]:
df.dropna(subset=["Borough"],axis=0,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


####Remove rows where Borough is not assigned.


In [7]:
df=df[df.Borough!="Not assigned"]
df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


####When the neighborhood is not assigned-> Neiborhood=Borough

In [8]:
for index, row in df.iterrows():
    if row['Neighborhood']=="Not assigned":
        df=df.replace(row['Neighborhood'],row['Borough'])
df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


####Group by PostalCode (with neighborhoods separated by comma)

In [9]:
grp=df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
grp.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
